# Capacitated Vehicle Routing Problem

## TODO:
- Some explanation and general info of the capacitated vehicle routing problem here. (Will be inserted later)
- Add external file content to notebook
- consider difference between cartesian and geographical coordinates (latitude and longitude are actually angle coordinates, not location coordinates)
- shorten notebook
- display dataframe with .head()
- add time window constraint

First, we import some useful python packages for solving the capacitated vehicle routing problem.

In [1]:
from __future__ import print_function
from six.moves import xrange
from IPython.display import display, HTML
import openrouteservice
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# Setting up the problem

In [2]:
CVRP = np.array([[  8.683448,  49.416961,   0.  ],
                 [  8.681247,  49.412388,  22.  ],
                 [  8.662789,  49.37446,    7.  ],
                 [  8.663488,  49.390504,  22.  ],
                 [  8.650365,  49.424738,   8.  ],
                 [  8.684574,  49.393819,  17.  ],
                 [  8.636446,  49.430018,   8.  ],
                 [  8.693395,  49.401669,  25.  ],
                 [  8.685109,  49.387783,  17.  ],
                 [  8.670162,  49.406377,   4.  ],
                 [  8.63245,   49.409997,  24.  ],
                 [  8.689162,  49.372064,  23.  ],
                 [  8.670682,  49.396678,   3.  ],
                 [  8.690306,  49.372096,  26.  ],
                 [  8.69358,   49.378182,  13.  ],
                 [  8.63606,   49.395647,   4.  ],
                 [  8.670398,  49.408443,  28.  ],
                 [  8.637578,  49.429881,  25.  ],
                 [  8.682342,  49.403657,   3.  ],
                 [  8.64455,   49.413006,   3.  ],
                 [  8.685483,  49.406815,  16.  ],
                 [  8.691885,  49.372719,  12.  ]])

df_CVRP = pd.DataFrame(CVRP, columns=['longitude', 'latitude', 'demand'])
df_CVRP.columns.name = 'node number'
display(df_CVRP)

node number,longitude,latitude,demand
0,8.683448,49.416961,0.0
1,8.681247,49.412388,22.0
2,8.662789,49.374460,7.0
3,8.663488,49.390504,22.0
4,8.650365,49.424738,8.0
5,8.684574,49.393819,17.0
6,8.636446,49.430018,8.0
7,8.693395,49.401669,25.0
8,8.685109,49.387783,17.0
9,8.670162,49.406377,4.0


# Solving the capacitated vehicle routing problem

To solve the capacitated vehicle routing problem, we will use the [**google or-tools**](https://github.com/google/or-tools) as well as the [**openrouteservie**](https://openrouteservice.org/). First, we use the matrix API of the openrouteservice, which will give us a symmetric distance or duration matrix for a list of locations/coordinates, where every location is paired with each other. For further explaination please refer to the [**documentation**](https://openrouteservice.org/documentation/#/reference/matrix/matrix). 

We start with querying a distance matrix from the API. The values of the matrix are given in meters.  

Note: If you skipped the previous chapter "Setting up the problem", please make sure that you have executed all previous cells in order for the following code to run without complications. 

In [3]:
api_key = '58d904a497c67e00015b45fc9298e8d961e64b48b066a43e51d39887'
client = openrouteservice.Client(key=api_key)

response = client.distance_matrix(locations=df_CVRP.iloc[:, :2].as_matrix().tolist(), metrics=['duration'])
duration_matrix = np.array(response['durations']).astype(int)
df_duration_matrix = pd.DataFrame(duration_matrix)
display(df_duration_matrix.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,147,635,383,515,392,561,423,404,248,...,338,617,662,703,250,643,307,487,296,659
1,147,0,523,271,403,280,513,344,292,136,...,226,505,550,590,138,531,195,375,184,547
2,585,473,0,304,699,438,652,510,400,416,...,300,366,496,541,434,781,341,530,377,408
3,412,300,251,0,526,265,609,337,277,243,...,127,439,535,445,261,604,168,388,204,480
4,530,418,755,503,0,512,575,576,524,337,...,458,737,782,609,298,264,427,393,416,779


As we can see the openrouteservice API returns a symmetric distance matrix with the diagonal being zeros, since the distance of every node to itself is zero. In the next step, we use the or-tool on the obtained distance matrix. For this purpose, we first initialize some variables and declare the or-tool solver:

In [4]:
locations = df_CVRP.iloc[:, :2].as_matrix()
demands = df_CVRP.iloc[:, 2].as_matrix()
num_locations = len(locations)
depot = 0    # The depot is the start and end point of each route.
num_vehicles = 4

# Create routing model.
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)

The SetArcCostEvaluatorOfAllVehicles() method of the or-tool requires a callable object. Therefore we have to wrap the distance matrix and the demands within a class with suitable methods that can be called by the SetArcCostEvaluatorOfAllVehicles() method. 

In [5]:
class matrix_obj(object):
    def __init__(self, matrix):
        self.matrix = matrix
    def Distance(self, from_node, to_node):
        return self.matrix[from_node][to_node]
    
class demand_obj(object):
    def __init__(self, demands):
        self.demands = demands
    def Demand(self, from_node, to_node):
        return self.demands[from_node]
    
# distance callback
get_duration = matrix_obj(duration_matrix).Distance
routing.SetArcCostEvaluatorOfAllVehicles(get_duration)

# demand callback
get_demand = demand_obj(demands).Demand

Routing problems involve quantities that accumulate along a vehicle's route. The routing solver stores each quantity of this type in an object called a dimension. We now need to add the demand as a new dimension to the routing model. We can do this with the AddDimension() method by providing the demand callback and the vehicle capacity, which is 100 in our case. For every location where a vehicle stops along its route, the total demand on the vehicle increases by the demand at that location.  

In [6]:
routing.AddDimension(get_demand, 0, 100, True, "Capacity")

True

In [7]:
search_parameters = pywrapcp.RoutingModel.DefaultModelParameters()
#search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC)

The last step is to solve the routing model and display the solution:

In [ ]:
assignment = routing.SolveWithParameters(search_parameters) # this line is responsible for jupyter kernel crash, reason unknown

In [10]:
if assignment:
    print("Total distance of all routes: " + str(assignment.ObjectiveValue()) + "\n")
    for vehicle_nbr in range(num_vehicles):
        index = routing.Start(vehicle_nbr)
        index_next = assignment.Value(routing.NextVar(index))
        route = ''
        route_dist = 0
        route_demand = 0

        while (not routing.IsEnd(index_next)):
            node_index = routing.IndexToNode(index)
            node_index_next = routing.IndexToNode(index_next)
            route += str(node_index) + " -> "
            # Add the distance to the next node.
            route_dist += dist_callback(node_index, node_index_next)
            # Add demand.
            route_demand += demands[node_index_next]
            index = index_next
            index_next = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        node_index_next = routing.IndexToNode(index_next)
        route += str(node_index) + " -> " + str(node_index_next)
        route_dist += dist_callback(node_index, node_index_next)
        print("Route for vehicle " + str(vehicle_nbr) + ":\n\n" + route + "\n")
        print("Distance of route " + str(vehicle_nbr) + ": " + str(route_dist))
        print("Demand met by vehicle " + str(vehicle_nbr) + ": " + str(route_demand) + "\n")
else:
    print('No solution found.')


No solution found.


In [12]:
help(pywrapcp.RoutingModel.SolveWithParameters(search_parameters))

TypeError: SolveWithParameters() missing 1 required positional argument: 'search_parameters'

# Adding Time Window Constraint

None
